# Reterive Flywheel QA Metadata for Inspection

Welcome! This is an introductory worksheet to explore how we can use Flywheel's SDK to retrieve quality assurance metrics stored in Flywheel from MRIQC gear! 

**Date modified:** 02/16/2025<br>
**Authors:** Amy Hegarty, Intermountain Neuroimaging Consortium

**Sections:**
1. IMPORT STATEMENTS
2. FLYWHEEL LOGIN
3. WEEKLY PHANTOM SCAN QUALTIY ASSURANCE
4. QUALITY ASSURANCE FOR MB=8 SEQUENCES
-----

Before starting...
1. Be sure you have configured your conda environment to view ics managed conda environments and packages. If you haven't get started [here](https://inc-documentation.readthedocs.io/en/latest/pl_and_blanca_basics.html#setting-up-conda-environments).

2. Be sure to select the `incenv` kernel from the list of available kernels. If you don't see the `incenv` kernel, contact Amy Hegarty <Amy.Hegarty@colorado.edu> or follow the instructions [here](https://inc-documentation.readthedocs.io/en/latest/pl_and_blanca_basics.html#setting-up-conda-environments) to setup a new kernel in a shared conda environment. 

## __IMPORT STATEMENTS__
Here we will load all packages used in the worksheet. This includes some custom helper functions stored in helper_functions.py

In [ ]:
import os, logging
import flywheel
import pandas as pd
from datetime import datetime, timedelta

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
logging.basicConfig(level=logging.WARNING)

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('main')


## Helper Function for getting date
def get_x_days_ago(x, date=None):
    if date is None:
        date = datetime.now()
    return date - timedelta(days=x)

## __FLYWHEEL LOGIN__
Be sure you have first logged into flywheel using the command line interface. Once you have stored your API key, you will not need to log in subsequent times. Follow instructions [here](https://inc-documentation.readthedocs.io/en/latest/cli_basics.html#cli-from-blanca-compute-node). 

In [ ]:
fw = flywheel.Client('')
fw.get_config().site.api_url

## __WEEKLY PHANTOM SCAN QUALTIY ASSURANCE__
Retrieve image quality metrics from weekly QA project in flywheel. Note: This could similarly be accomplished in FlyQL or Flywheel DataViews. Once all relevant metadata is pulled from Flywheel, make pretty plots.

In [ ]:
# USER INPUTS!
project_path = "<group/project>"   ## e.g.  project_path = "mbanich/ABCDQA"
# create lookback window
created_by = get_x_days_ago(180).strftime('%Y-%m-%d')

# filtered sessions for specific project path
filtered_sessions=fw.get(fw.lookup(project_path).id).sessions.find(f'created>{created_by}')

iqm = pd.DataFrame()
# IQM data are stored as file metadata (nifti files inside acqusition containers)
# loop though sessions --> acquisitons --> files to retrieve all IQM data. 
for session in filtered_sessions:
    for acq in session.acquisitions():
        if "func-bold" in acq.label and "ignore-BIDS" not in acq.label:
            full_acq = fw.get_acquisition(acq.id)
            for f in full_acq.files:
                if f.type == 'nifti':
                    if "IQM" in f.info:
                        # store all IQMS
                        vals = f.info['IQM']
                        
                        # add some extra metadata from nifti header
                        vals["IOPD1"] = f.info['ImageOrientationPatientDICOM'][0]   # patient oritation in scanner (can affect snr)
                        vals["IOPD2"] = f.info['ImageOrientationPatientDICOM'][1]
                        vals["IOPD3"] = f.info['ImageOrientationPatientDICOM'][2]
                        vals["IOPD4"] = f.info['ImageOrientationPatientDICOM'][3]
                        vals["IOPD5"] = f.info['ImageOrientationPatientDICOM'][4]
                        vals["IOPD6"] = f.info['ImageOrientationPatientDICOM'][5]
                        vals["MB"] = f.info['MultibandAccelerationFactor'] if 'MultibandAccelerationFactor' in f.info else None  # multi-band acceleration fact (affects snr big time!)
                        vals["TR"] = f.info['RepetitionTime']
                        
                        # add some extra descriptors of data including timestamp, acquisition name, subject and session ids
                        vals["timestamp"] = f.created.strftime('%Y-%m-%d')   
                        vals["session"] = session.label
                        vals["subject"] = fw.get(session.parents["subject"]).label
                        vals["project"] = fw.get(session.parents["project"]).label
                        vals["acquisition"] = acq.label
                        
                        # store in master sheet (dataframe -- use later for plotting...)
                        df = pd.DataFrame(vals, index=[0])
                        iqm = pd.concat([iqm, df], ignore_index=True)

In [ ]:
# plot snr as scatterplot
g = sns.relplot(data=iqm, x="timestamp",y="snr",hue="acquisition")
xlims_orig = g.ax.get_xlim()
xshading = [g.ax.get_xlim()[0], g.ax.get_xlim()[1]-2]

legend_handles = g._legend.legendHandles
colors = [h.get_facecolor() for h in legend_handles]

# add mean and 95% confidence intervals 
for idx, i in enumerate([f.get_text() for f in g._legend.texts]):
    
    # plot bar for each acquisiton type
    data = iqm.loc[(iqm["acquisition"] == i), "snr"]
    mean = np.mean(data)

    ci = stats.t.interval(alpha=0.95, df=len(data)-1, 
              loc=np.mean(data), 
              scale=stats.sem(data)) 

    # Create the plot
    sns.lineplot(x=xshading, y=[mean, mean], color=colors[idx].flatten(), legend=False)
    sns.lineplot(x=[xshading[1],xlims_orig[1]], y=[mean, mean], color=colors[idx].flatten(), legend=False, linestyle=':')
    plt.fill_between(xshading, ci[0], ci[1], color=colors[idx].flatten(), alpha=0.05)
    
# make pretty and save image
g.ax.set_xlim(xlims_orig)
plt.xticks(rotation=90)
os.makedirs('plots',exist_ok = True)
plt.savefig("plots/phantom_QA{}.png".format(datetime.now().strftime('-%Y-%m-%d')), bbox_inches='tight')  

## __QUALITY ASSURANCE FOR MB=8 SEQUENCES__

In [ ]:
# create lookback window across all projects
created_by = get_x_days_ago(30).strftime('%Y-%m-%d')
filtered_sessions=fw.sessions.find(f'created>{created_by}')

# IQM data are stored as file metadata (nifti files inside acqusition containers)
# loop though sessions --> acquisitons --> files to retrieve all IQM data. 
iqm = pd.DataFrame()
for session in filtered_sessions:
    
    # skip sandbox projects
    if "sandbox" in fw.get(session.parents["project"]).label:
        continue
    for acq in session.acquisitions():
        if "func-bold" in acq.label and "ignore-BIDS" not in acq.label:
            full_acq = fw.get_acquisition(acq.id)
            for f in full_acq.files:
                if f.type == 'nifti':
                    if "IQM" in f.info:
                         # store all IQMS
                        vals = f.info['IQM']
                        
                        # add some extra metadata from nifti header
                        vals["IOPD1"] = f.info['ImageOrientationPatientDICOM'][0]   # patient oritation in scanner (can affect snr)
                        vals["IOPD2"] = f.info['ImageOrientationPatientDICOM'][1]
                        vals["IOPD3"] = f.info['ImageOrientationPatientDICOM'][2]
                        vals["IOPD4"] = f.info['ImageOrientationPatientDICOM'][3]
                        vals["IOPD5"] = f.info['ImageOrientationPatientDICOM'][4]
                        vals["IOPD6"] = f.info['ImageOrientationPatientDICOM'][5]
                        vals["MB"] = f.info['MultibandAccelerationFactor'] if 'MultibandAccelerationFactor' in f.info else None  # multi-band acceleration fact (affects snr big time!)
                        vals["TR"] = f.info['RepetitionTime']
                        
                        # add some extra descriptors of data including timestamp, acquisition name, subject and session ids
                        vals["timestamp"] = f.created.strftime('%Y-%m-%d')   
                        vals["session"] = session.label
                        vals["subject"] = fw.get(session.parents["subject"]).label
                        vals["project"] = fw.get(session.parents["project"]).label
                        vals["acquisition"] = acq.label
                        
                        # store in master sheet (dataframe -- use later for plotting...)
                        df = pd.DataFrame(vals, index=[0])
                        iqm = pd.concat([iqm, df], ignore_index=True)



In [ ]:
# plot snr across all sessions.
sns.relplot(data=iqm[iqm['MB']==8], x="timestamp",y="snr",hue="project")
plt.xticks(rotation=90)

os.makedirs('plots',exist_ok = True)
plt.savefig("plots/mb8_QA{}.png".format(datetime.now().strftime('-%Y-%m-%d')), bbox_inches='tight')  

In [ ]:
# WARNING!! THIS CELL DELETES DATA

import shutil
# cleanup your outputs when you are finished...
shutil.rmtree('plots')